## 🛠 Introduction

In this notebook, we’ll build a **collaborative multi-agent article generation system** powered by Azure OpenAI and AutoGen’s agentchat framework, exposed via a minimalist Flask API. Each agent plays a defined editorial role—title generation, content creation, formatting, and format-checking—working together to produce a polished Markdown article file.

### Core Technologies

- **autogen-agentchat** & **autogen_ext[openai,azure]**  
  Multi-agent orchestration library for Azure OpenAI models.
- **autogen_core**  
  Core runtime, message routing, and tool integration.
- **Flask**  
  Lightweight web framework to expose a `/generate-article` endpoint.
- **rich**  
  Nicely formatted console output (Markdown rendering in notebook).

---

## What We’ll Do in This Notebook

We divide the pipeline into clear, progressive steps. By the end, you’ll have a running Flask service that, given a topic, spins up a group‐chat among agents to generate, format, verify, and save an article Markdown file.

### Step 1: Install Dependencies  

```bash
pip install -qU autogen-agentchat autogen-ext[openai,azure] flask autogen_core rich
```

This brings in the AutoGen agentchat extensions, Flask, and Rich for pretty output.

### Step 2: Configure Environment & Model Client

1. Set Azure OpenAI environment variables:

   * `AZURE_OPENAI_DEPLOYMENT_NAME`
   * `OPENAI_API_VERSION`
   * `AZURE_OPENAI_API_KEY`
   * `AZURE_OPENAI_ENDPOINT`

2. Initialize an `AzureOpenAIChatCompletionClient` for model calls.

---

### Step 3: Define the “Generate Title” Tool

* **Function**: `generate_title(topic: str) → str`
* Sends a prompt to the model to create a concise, catchy headline.
* Registered as a `FunctionTool` so agents can invoke it during content creation.

---

### Step 4: Load Article Template

* Read in `article_template.txt` from disk.
* This Markdown skeleton guides the ContentCreator and Formatter agents.

---

### Step 5: Build Base Agent Classes

* **BaseGroupChatAgent**

  * Wraps common chat history management, message handlers, and console output.
  * Routes two message types:

    * `GroupChatMessage` for plain user/model content.
    * `RequestToSpeak` for turn-taking.

---

### Step 6: Implement Specialized Agents

1. **ContentCreatorAgent**

   * Persona: “Content Creator”
   * First invokes the `generate_title` tool, then writes the article into the provided template.
2. **FormatterAgent**

   * Persona: “Formatter”
   * Rewrites raw content to adhere precisely to the Markdown template.
3. **FormatCheckerAgent**

   * Persona: “Format Checker”
   * Validates template compliance; responds with `APPROVED` or suggestions.

Each agent extends `BaseGroupChatAgent`, supplies its own `system_message`, and (for ContentCreator) wires in the title-generation tool.

---

### Step 7: Orchestrate with GroupChatManager

* Tracks conversation history and enforces save logic when “APPROVED” is received.
* Decides which agent speaks next by prompting the model with:

  * Available roles
  * Conversation transcript
  * Instructions to respond with exactly one role name.
* On approval, writes the final Markdown file (`<topic>.md`) and stops the runtime.

---

### Step 8: Expose as a Flask Endpoint

* **`POST /generate-article?topic=…`**

  1. Spawns a background thread running `_run_group_chat(topic)`.
  2. Blocks until the event signals completion.
  3. Returns a JSON response with the filename of the generated article.

Under the hood, `_run_group_chat`:

1. Instantiates a `SingleThreadedAgentRuntime`.
2. Registers all agents and their subscriptions on the `"group_chat"` topic.
3. Publishes the initial UserMessage:

   > “Write a professional article on ‘<topic>’ using the given template.”
4. Awaits idle, then closes the model client.

---

## By the End of This Notebook

* A **Flask-powered API** that, given any topic, generates a full Markdown article.
* A **multi-agent workflow** illustrating:

  * Tool invocation within agentchat,
  * Turn-taking logic,
  * Role-based system prompts,
  * Automated title generation, formatting, and quality checks.
* A saved file `"<topic>.md"` containing your approved article.

This pipeline showcases how to combine **Azure OpenAI**, **AutoGen agentchat**, and a simple **web interface** to automate end-to-end content creation tasks.


## Step 1: Install Dependencies

Before running any code, we need to install all required Python packages. This single combined command:

* **Sets up the AutoGen agentchat framework** (`autogen-agentchat`) and **Azure/OpenAI extensions** (`autogen-ext[openai,azure]`).
* **Installs Flask** for providing an HTTP API, the **AutoGen core** runtime, and **Rich** for beautifully formatted console logs.

By using `-qU`, we ensure quiet output and upgrade to the latest versions.

In [ ]:
# Install AutoGen agent framework with OpenAI and Azure support
!pip install -qU "autogen-agentchat" "autogen-ext[openai,azure]"

In [ ]:
# Install Flask (API server), AutoGen core, and Rich (for better console output)
!pip install -qU flask autogen_core rich

## Step 2: Import Libraries & Configure Model Client

In this cell, we:

1. **Import standard libraries** for async execution (`asyncio`), threading (`Thread`), JSON handling, UUID generation, and URL encoding.
2. **Bring in Jupyter utilities** (`display`, `ipywidgets`) for potential interactive input.
3. **Load Flask** for the web endpoint.
4. **Define Pydantic models** and **AutoGen core classes** for message routing and agent orchestration.
5. **Configure Azure OpenAI credentials** via environment variables. Replace placeholders with your actual key and endpoint.
6. **Instantiate** an `AzureOpenAIChatCompletionClient` to handle all subsequent LLM calls.

This setup cell forms the backbone of our multi-agent system, ensuring all dependencies and credentials are initialized before any logic runs.

In [ ]:
# Standard libraries for OS, JSON, UUIDs, async, threading, and typing
import os
import json
import uuid
import asyncio
import urllib.parse
from threading import Thread
from typing import List


# Flask for API creation
from flask import Flask, request, jsonify

# Typing extensions and data validation
from typing_extensions import Annotated
from pydantic import BaseModel

# AutoGen core components for agent setup and messaging
import autogen_core
from autogen_core import (
    DefaultTopicId, MessageContext, SingleThreadedAgentRuntime, TopicId,
    TypeSubscription, message_handler, RoutedAgent, CancellationToken
)

# Message and model handling in AutoGen
from autogen_core.models import (
    SystemMessage, UserMessage, AssistantMessage, ChatCompletionClient,
    LLMMessage, FunctionExecutionResult, FunctionExecutionResultMessage
)

# Tool wrapper and Azure OpenAI client
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Rich output for CLI rendering
from rich.console import Console
from rich.markdown import Markdown


In [ ]:
# Set environment variables for Azure OpenAI configurationos.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-mini"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-mini"          # Azure model deployment name
os.environ["OPENAI_API_VERSION"] = "2024-12-01-preview"             # API version for Azure OpenAI
os.environ["AZURE_OPENAI_API_KEY"] = "6qZqBQ5RB7ImYsP7ajBcIX7rZUnD22vTz76QI5R3FWlWAJ0EjvcXJQQJ99BEACHYHv6XJ3w3AAAAACOGdXrz"                             # Your Azure OpenAI API key (fill this)
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ai-hpesgalite4096ai784823033224.openai.azure.com"                            # Azure OpenAI endpoint URL (fill this)

# Initialize the Azure OpenAI chat model client
model_client = AzureOpenAIChatCompletionClient(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],              # Deployment name
    api_base=os.environ["AZURE_OPENAI_ENDPOINT"],                  # Endpoint URL
    api_type="azure",                                              # API type
    api_version=os.environ["OPENAI_API_VERSION"],                  # API version
    api_key=os.environ["AZURE_OPENAI_API_KEY"]                     # API key
)

## Step 3: Initialize Flask Application

Here we create the Flask application instance. It will later host our `/generate-article` endpoint, receiving HTTP POST requests to trigger the article generation pipeline.


In [ ]:
# Initialize the Flask web application
app = Flask(__name__)

## Step 4: Define Title-Generation Tool

We build a reusable asynchronous function `generate_title`:

* **Input**: A plain string topic.
* **Action**: Crafts a prompt instructing the model to generate a clean, catchphrase-style headline.
* **Output**: Returns the generated title text.

This function is then wrapped in `FunctionTool`, which allows agents to invoke it as a first-class “tool” within the chat flow.

In [ ]:
# Async function to generate an article title using LLM
async def generate_title(topic: Annotated[str, "The user-provided article topic"]) -> str:
    prompt = f"""You are a headline expert. Generate a professional, concise, and catchy article title for the topic below.

    Topic: {topic}

    Only return the title text. Do not include quotes, labels, or extra commentary."""

    # Call the LLM to get a title based on the prompt
    completion = await model_client.create(
        messages=[UserMessage(content=prompt, source="user")]
    )
    return completion.content.strip()  # 🧹 Return clean title text

# 🛠 Register the function as a tool for use in AutoGen workflows
generate_title_tool = FunctionTool(
    generate_title,
    description="Use LLM to generate a professional article title based on the input topic."
)

## Step 5: Load Article Template & Setup Console

We read our predefined Markdown skeleton (`article_template.txt`), which defines headings and placeholders for the final article. Simultaneously, we tear up Rich’s `Console` for live Markdown rendering as the pipeline runs.

In [ ]:
# Load the article template from file
with open("article_template.txt", "r") as f:
    article_template = f.read()

# Initialize Rich console for pretty terminal output
console = Console()


## Step 6: Define Message Schemas

Pydantic models help enforce structure for messages exchanged between agents:

* **GroupChatMessage**: Wraps a `UserMessage` body when broadcasting agent content.
* **RequestToSpeak**: A simple signal type indicating it's an agent’s turn.

In [ ]:
# Define a message wrapper for group chat communication
class GroupChatMessage(BaseModel):
    body: UserMessage  # Holds a user-generated message

# Define a signal model to request speaking in group chat
class RequestToSpeak(BaseModel):
    pass  # Used as a trigger with no additional data


## Step 7: Build Base Agent Class

`BaseGroupChatAgent` centralizes shared behaviors:

1. **Initialization**: Accepts a role description, topic type, LLM client, and persona system message.
2. **Message Handling**: On receiving a chat message, it appends both a system-transfer note and the content to its history.
3. **Speaking**: When prompted to speak, it prints its header, sends the full history (including its system prompt) to the LLM, records the response, and broadcasts it back to the group.

This scaffolding underpins all specialized agents.

In [ ]:
# Define a base group chat agent using AutoGen's RoutedAgent
class BaseGroupChatAgent(RoutedAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient, system_message: str) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type  # Group topic category/type
        self._model_client = model_client                    # Azure/OpenAI model client
        self._system_message = SystemMessage(content=system_message)  # Initial persona setup
        self._chat_history: List[LLMMessage] = []            # Track conversation history

    # Handle incoming user messages routed to this agent
    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        self._chat_history.extend([
            UserMessage(content=f"Transferred to {message.body.source}", source="system"),  # System note
            message.body  # Add the actual user message
        ])

    # Handle a request to speak event (e.g., when the agent is chosen to respond)
    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        console.print(Markdown(f"### {self.id.type}: "))  # Print who is speaking
        # Add a system instruction to assume persona
        self._chat_history.append(UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system"))

        # Query the model with system message + history
        completion = await self._model_client.create([self._system_message] + self._chat_history)
        assert isinstance(completion.content, str)

        # Store and display assistant's response
        self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))
        console.print(Markdown(completion.content))

        # Publish the assistant's message to the group chat
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=completion.content, source=self.id.type)),
            topic_id=DefaultTopicId(type=self._group_chat_topic_type)
        )


## Step 8: Implement Specialized Agents

We extend `BaseGroupChatAgent` to define three distinct roles:

1. **ContentCreatorAgent**:

   * Uses `generate_title_tool` to create the headline.
   * Writes the full article within the provided Markdown structure.
2. **FormatterAgent**:

   * Reprocesses raw content to strictly conform to the template’s layout.
3. **FormatCheckerAgent**:

   * Reviews final output, returning `APPROVED` or specific formatting suggestions.

Each agent sets its own system prompt and tools, then leverages the shared `handle_request_to_speak` flow.

In [ ]:
# Agent responsible for generating article title and content
class ContentCreatorAgent(BaseGroupChatAgent):
    def __init__(self, group_chat_topic_type: str, model_client: ChatCompletionClient):
        # Initialize with a system message and markdown template instruction
        super().__init__(
            "Content Creator",  # Agent description
            group_chat_topic_type,
            model_client,
            f"""You are a content creator. First, use the `generate_title` tool to come up with a professional article title. Then write a full article using the provided markdown structure:\n{article_template}"""
        )
        self._tools = [generate_title_tool]  # Register the title generation tool

    # 🎙️ Respond when asked to speak in the group chat
    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        console.print(Markdown(f"### {self.id.type}: "))  # Print agent role visibly
        self._chat_history.append(
            UserMessage(content="Generate the article starting with title generation.", source="system")
        )

        # 🔄 First attempt: let LLM decide to generate or invoke a tool
        completion = await self._model_client.create(
            messages=[self._system_message] + self._chat_history,
            tools=self._tools,
            cancellation_token=ctx.cancellation_token
        )

        if isinstance(completion.content, str):
            # 🧾 LLM directly responded with article content
            self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))
        else:
            # 🔧 Tool call path: LLM invoked the title generation tool
            tool_call = completion.content[0]
            arguments = json.loads(tool_call.arguments)
            tool_result = await self._tools[0].run_json(arguments, ctx.cancellation_token)

            print(f"🏷️ Generated Title: {tool_result.strip()}")

            # Add the tool call and result to the chat history
            self._chat_history.append(
                AssistantMessage(content=[tool_call], source=self.id.type)
            )
            self._chat_history.append(
                FunctionExecutionResultMessage(content=[
                    FunctionExecutionResult(
                        call_id=tool_call.id,
                        content=tool_result,
                        is_error=False,
                        name=self._tools[0].name
                    )
                ])
            )

            # Re-run LLM after tool execution to generate article body
            completion = await self._model_client.create(
                messages=[self._system_message] + self._chat_history,
                cancellation_token=ctx.cancellation_token
            )
            self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))

        # Display the final article content
        console.print(Markdown(self._chat_history[-1].content))

        # Broadcast the final message to the group chat
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=self._chat_history[-1].content, source=self.id.type)),
            topic_id=DefaultTopicId(type=self._group_chat_topic_type)
        )

In [ ]:
# Agent to format content using the predefined article markdown template
class FormatterAgent(BaseGroupChatAgent):
    def __init__(self, group_chat_topic_type: str, model_client: ChatCompletionClient):
        super().__init__(
            "Formatter",  # Agent role/description
            group_chat_topic_type,
            model_client,
            f"You are a formatter. Format the content into the expected markdown layout using the following template:\n{article_template}"
        )

# Agent to check if the article formatting follows the expected markdown structure
class FormatCheckerAgent(BaseGroupChatAgent):
    def __init__(self, group_chat_topic_type: str, model_client: ChatCompletionClient):
        super().__init__(
            "Format Checker",  # Agent role/description
            group_chat_topic_type,
            model_client,
            "You are a format checker. Verify if the article is correctly formatted as per the template. Respond only with APPROVED or specific SUGGESTIONS."
        )

## Step 9: Orchestrate with GroupChatManager

`GroupChatManager` dictates which agent speaks next and handles finalization:

* **Conversation Tracking**: Maintains a full transcript.
* **Save Logic**: Detects `APPROVED` and writes the complete article to `<topic>.md`.
* **Turn-Taking**: Prompts the LLM to choose the next role from the remaining participants.

This central controller ensures smooth collaboration and triggers shutdown upon completion.

In [ ]:
# GroupChatManager orchestrates the agent conversation flow
class GroupChatManager(RoutedAgent):
    def __init__(self, participant_topic_types: List[str], model_client: ChatCompletionClient, participant_descriptions: List[str], topic: str) -> None:
        super().__init__("Group Chat Manager")  # Agent name
        self._participant_topic_types = participant_topic_types        # List of agent types (e.g., ContentCreator, Formatter)
        self._model_client = model_client                              # LLM client
        self._chat_history: List[UserMessage] = []                     # Tracks user messages in the chat
        self._participant_descriptions = participant_descriptions      # Descriptions for each agent
        self._previous_participant_topic_type: str | None = None       # Prevent immediate repetition
        self._topic = topic                                            # Topic of the article
        self._last_complete_article = ""                               # Stores article before approval

    # Handles incoming messages from agents
    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        self._chat_history.append(message.body)

        if isinstance(message.body.content, str):
            content = message.body.content.strip()
            print(f"🔹 Received content from {message.body.source}: {content[:60]}...")

            # Store completed article before final approval
            if content.endswith("End of the article."):
                self._last_complete_article = content
                print("✅ Stored complete article content in memory.")

            # If format checker approves, save the article to file
            elif content == "APPROVED" and self._last_complete_article:
                base_filename = self._topic.strip() or "article"
                with open(f"{base_filename}.md", "w") as f:
                    f.write(self._last_complete_article)
                print(f"📄 Saved approved article to '{base_filename}.md'.")

                # 🔚 Stop runtime gracefully (if set)
                if self._runtime:
                    print("🛑 Stopping runtime after approval.")
                    # self._runtime.request_termination()  # Optional: explicitly stop if runtime support exists
                else:
                    print("⚠ Runtime reference is not set; cannot stop runtime.")
                return

        # Decide next speaking agent based on chat history
        history = "\n".join([f"{m.source}: {m.content}" for m in self._chat_history])
        roles = "\n".join([
            f"{t}: {d}" for t, d in zip(self._participant_topic_types, self._participant_descriptions)
            if t != self._previous_participant_topic_type  # Avoid back-to-back same agent
        ])

        prompt = (
            "You are managing a collaborative article generation task.\n"
            f"{roles}\n\nConversation:\n{history}\n\n"
            "Select the next role to speak (respond ONLY with one of: ContentCreator, Formatter, FormatChecker)."
        )

        system_message = SystemMessage(content=prompt)

        # Ask model to choose next agent
        completion = await self._model_client.create([system_message], cancellation_token=ctx.cancellation_token)
        response = completion.content.strip()
        print(f"🔍 Model suggested next role: '{response}'")

        # Validate and trigger the next speaker
        selected_topic_type = next(
            (t for t in self._participant_topic_types if t.lower() == response.lower()), None
        )

        if selected_topic_type:
            self._previous_participant_topic_type = selected_topic_type
            await self.publish_message(RequestToSpeak(), DefaultTopicId(type=selected_topic_type))
            print(f"➡ Published RequestToSpeak to: {selected_topic_type}")
        else:
            print(f"❌ Invalid role selected by model: '{response}' — skipping this round.")


## Step 10: Expose as Flask Endpoint

We define `/generate-article` that:

1. **Spawns** a background thread calling `_run_group_chat(topic)`.
2. **Blocks** until the thread signals completion via an event.
3. **Returns** JSON containing the output filename.

The helper `_run_group_chat`:

* Registers all agents and subscriptions,
* Publishes the initial user request,
* Waits for the runtime to idle,
* Closes the LLM client.

In [ ]:
# 📡 Flask route to trigger article generation via HTTP POST
@app.route("/generate-article", methods=["POST"])
def generate_article():
    # Extract the article topic from query params or fallback to default
    topic = request.args.get("topic") or request.args.get("T") or "Benefit of AI in Healthcare"
    topic = topic.strip()

    # Event to block response until background generation finishes
    import threading
    done_event = threading.Event()

    # Launch group chat in a background thread
    def run_generation():
        asyncio.run(_run_group_chat(topic))  # Run async function inside thread
        done_event.set()  # Notify completion

    thread = Thread(target=run_generation)
    thread.start()

    # ⏱Wait until the generation thread completes
    done_event.wait()

    filename = f"{topic}.md"
    return jsonify({
        "status": "done",
        "message": "Article generated",
        "file": filename,
        "topic": topic
    })

# Asynchronous orchestration of agents for article generation
async def _run_group_chat(topic):
    runtime = SingleThreadedAgentRuntime()  # Single-threaded AutoGen runtime
    participants = ["ContentCreator", "Formatter", "FormatChecker"]  # Agent types
    descriptions = [
        "Creates initial article content.",
        "Formats the article.",
        "Checks article formatting."
    ]

    # Register each agent in the runtime
    await ContentCreatorAgent.register(runtime, "ContentCreator", lambda: ContentCreatorAgent("group_chat", model_client))
    await FormatterAgent.register(runtime, "Formatter", lambda: FormatterAgent("group_chat", model_client))
    await FormatCheckerAgent.register(runtime, "FormatChecker", lambda: FormatCheckerAgent("group_chat", model_client))
    await GroupChatManager.register(runtime, "GroupChatManager", lambda: GroupChatManager(participants, model_client, descriptions, topic))

    # Subscribe each agent to relevant topic types for message routing
    for t in participants:
        await runtime.add_subscription(TypeSubscription(topic_type=t, agent_type=t))           # Direct topic
        await runtime.add_subscription(TypeSubscription(topic_type="group_chat", agent_type=t)) # Shared group chat
    await runtime.add_subscription(TypeSubscription(topic_type="group_chat", agent_type="GroupChatManager"))

    runtime.start()  # Start message loop
    session_id = str(uuid.uuid4())  # Generate unique session

    # 📨 Trigger initial article generation instruction
    await runtime.publish_message(
        GroupChatMessage(
            body=UserMessage(
                content=f"Write a professional article on '{topic}' using the given template.",
                source="User",
            )
        ),
        TopicId(type="group_chat", source=session_id),
    )

    await runtime.stop_when_idle()  # Stop when all agents are idle
    await model_client.close()      # Clean up model client connection


## Step 11: Run Flask Server & Test

Finally, we launch Flask in a daemon thread and trigger the pipeline via `curl`. The notebook prints each step’s output and any errors, concluding with the saved Markdown filename.

In [ ]:
# Function to launch Flask server
def run_flask():
    print("Starting Flask server at http://localhost:5001 ...")
    app.run(port=5001, debug=False, use_reloader=False)  # Start Flask app on port 5001

# Run Flask server in a background daemon thread
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True  # Ensure thread exits when main program ends
flask_thread.start()        # Start the server thread


In [ ]:
# Prompt user to enter an article topic
article_topic = input("Enter the topic of the article: ").strip()

# URL-encode the topic to safely pass it as a query parameter
encoded_topic = urllib.parse.quote(article_topic)

# Make a POST request to the Flask endpoint to trigger article generation
!curl -X POST "http://localhost:5001/generate-article?topic={encoded_topic}" -H "Content-Type: application/json"

# Inform the user that the article has been saved
print(f"\n✅ Article has been saved as '{article_topic}.md'")


## Step 12 : Deployment

We will now package this application as a Docker container to make it ready for deployment to any service of our choice.

PLEASE NOTE - You can skip the following section if you are using Google Colab, the following section requires a virutal machine(VM)/workstation with Docker setup complete in order to proceed

Now, we will create the Dockerfile to package and make the application deployment ready. We have already prepared the Dockerfile for you so you do not need to run the cell below.
At this point, close this notebook and open a Terminal window in your JupyterLab environment and follow the steps below - 
1. Enter the following command to run our Docker container
```bash
    docker build -t .
    docker run -p 5002:5002 -v $(pwd)/output:/app/output autogen-agent
```
2. Now open another terminal window and send a cURL request to our exposed endpoint
```bash
    curl -X POST "http://localhost:5002/generate-article?topic=AI%20in%20Space" -H "Content-Type: application/json"
```
Given below is the Dockerfile we are using, you do not need to execute this cell

In [ ]:
# Use official Python runtime base image
FROM python:3.11-slim

# Set environment variables
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Set working directory in the container
WORKDIR /app

# Copy requirements and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy app source code
COPY output .
COPY app.py .
COPY article_template.txt .

# Expose the port Flask runs on
EXPOSE 5002

# Run the app
CMD ["python", "app.py"]


## Summary

This notebook demonstrates how to build a fully automated, multi-agent article generator using Azure OpenAI and AutoGen’s **agentchat** framework, exposed via a simple Flask API. The workflow consists of:

1. **Environment Setup**  
   Install & import all necessary packages (AutoGen agentchat, Azure/OpenAI extensions, Flask, core runtime, Rich).

2. **Model Configuration**  
   Set Azure OpenAI credentials and instantiate a chat completion client for interacting with the LLM.

3. **Tool Definition**  
   Create an asynchronous `generate_title` function wrapped as a `FunctionTool` to produce professional article headlines.

4. **Template & Messaging**  
   Load a Markdown article template and define Pydantic schemas for structured inter-agent messaging (`GroupChatMessage`, `RequestToSpeak`).

5. **Agent Infrastructure**  
   Implement `BaseGroupChatAgent` to handle message routing, history tracking, and LLM calls; extend it to three specialized agents:
   - **ContentCreatorAgent**: Generates title and article body.
   - **FormatterAgent**: Formats content to match the template.
   - **FormatCheckerAgent**: Validates final formatting.

6. **Orchestration**  
   Use `GroupChatManager` to manage turn-taking, collect the full transcript, detect approval, save the final Markdown file, and stop the runtime.

7. **API Exposure & Execution**  
   Define a Flask endpoint `/generate-article` that spawns the agent runtime in a background thread, blocks until completion, and returns the generated filename. Finally, launch the server and test the full pipeline via a `curl` request, resulting in a saved `<topic>.md` file.


## Lessons Learned & Future Applications

- **Clear Role Separation**  
  Defining distinct agent personas (title‐creator, content‐writer, formatter, checker) ensures each component has a single responsibility, simplifying development and testing.

- **Tool-Based LLM Integration**  
  Wrapping LLM prompts as `FunctionTool` objects provides a structured, reusable interface for agents to invoke model capabilities safely.

- **Flexible Orchestration Logic**  
  The `GroupChatManager` pattern—using transcripts and LLM prompts to decide turn-taking—can adapt to any multi-step workflow requiring dynamic hand-offs.

- **Async + Web Server Synergy**  
  Combining an asynchronous agent runtime with a Flask endpoint and background threads shows how to expose conversational pipelines as easy-to-call HTTP services.

### Possible Extensions

- **Automated Data Reporting**  
  Agents for data ingestion, chart creation, narrative writing, and format validation can generate end-of-period business or research reports automatically.

- **Support Ticket Handling**  
  Chain agents for intent classification, response drafting, sentiment analysis, and escalation, delivering a fully automated support workflow.

- **Code Generation & Review**  
  Implement agents for snippet generation, linting, test creation, and compliance checks to streamline and automate parts of the software development lifecycle.

By applying this modular, tool-oriented, orchestrated pattern, you can rapidly build domain-specific multi-agent pipelines that leverage LLMs for a wide variety of tasks.  
